In [544]:
import uproot
import awkward as ak
import numpy as np
import math
import hist
import matplotlib.pyplot as plt
import os
import subprocess
import vector

In [545]:
vector.register_awkward() 

In [562]:
DATATYPE="data"
assert((DATATYPE=="mc") or (DATATYPE=="data"))
BASEDIR="/data" # basedir where to look for runXXX.DATATYPE.root files
IS_MC=True if DATATYPE=="mc" else False

In [563]:
def data_url(run,is_mc):
    basesrc="https://cernbox.cern.ch/remote.php/dav/public-files/VJZ5whMyF5Kxldd/dimuonData_LHC18m"
    if is_mc==True:
        basesrc+="MC"
    return f"{basesrc}/{run}/AnalysisResults.root"

def data_file_path(run,is_mc=IS_MC,dest=BASEDIR):
    datatype="mc" if is_mc else "data"
    print({dest},"/run",{run},".",{datatype},".root")
    return f"{dest}/run{run}.{datatype}.root"

def copy_cmd(run,is_mc,dest):
    return f"curl '{data_url(run,is_mc)}' > {data_file_path(run,is_mc,dest)}"

In [564]:
#SAMPLE_RUNS=[291694,291399]
SAMPLE_RUNS=[290223,291694,291399]
for run in SAMPLE_RUNS:
    print("Is it MC? Answer:", IS_MC);
    if not os.path.exists(data_file_path(run,IS_MC)):
        os.makedirs(BASEDIR,exist_ok=True)
        subprocess.call(copy_cmd(run,IS_MC,BASEDIR),shell=True)

Is it MC? Answer: False
{'/data'} /run {290223} . {'data'} .root
Is it MC? Answer: False
{'/data'} /run {291694} . {'data'} .root
Is it MC? Answer: False
{'/data'} /run {291399} . {'data'} .root


In [565]:
file = uproot.open(data_file_path(SAMPLE_RUNS[0],IS_MC))
events = file["eventsTree"]
events.show()

{'/data'} /run {290223} . {'data'} .root
name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
runNumber            | int32_t                  | AsDtype('>i4')
xVtx                 | double                   | AsDtype('>f8')
yVtx                 | double                   | AsDtype('>f8')
zVtx                 | double                   | AsDtype('>f8')
isCINT               | bool                     | AsDtype('bool')
isCMSL               | bool                     | AsDtype('bool')
isCMSH               | bool                     | AsDtype('bool')
isCMLL               | bool                     | AsDtype('bool')
isCMUL               | bool                     | AsDtype('bool')
nMuons               | int32_t                  | AsDtype('>i4')
Muon_E               | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
Muon_Px              | std::vector<float>       | AsJagged(As

In [566]:
events.num_entries

77444

# 1-Analyse en muon simple

In [567]:
def getTracks(events):
    return ak.zip({"px":events["Muon_Px"],
                       "py":events["Muon_Py"],
                       "pz":events["Muon_Pz"],
                       "E":events["Muon_E"],
                       "charge":events["Muon_Charge"],
                       "thetaAbs":events["Muon_thetaAbs"],
                       "matched":events["Muon_matchedTrgThreshold"]},
                    with_name='Momentum4D')

def getPairs(tracks):
    """ Extract the pairs from an array of tracks.
    
    And compute a few fields for each pair : their rapidity, mass and 
    whether or not the charge of their two tracks are different 
    """
    
    twoOrMore = tracks[ak.num(tracks)>=2]
    pairs = ak.combinations(twoOrMore,2,fields={"first","second"})
    one, two = pairs.slot0,pairs.slot1
    opposite_charge = one.charge != two.charge
    s = (one+two)
    pairs["y"]=s.rapidity
    pairs["pt"]=s.pt
    pairs["unlike-sign"]=opposite_charge
    pairs["mass"]=s.mass
    return pairs

In [568]:
def scan(dataDescription, 
              hMag:hist.Hist, hTheta:hist.Hist, hEta:hist.Hist, hPt:hist.Hist, hPhi:hist.Hist, 
              hpMinv:hist.Hist, hpY:hist.Hist, hpPt:hist.Hist,
              eventSelector=lambda x:[True]*len(x),
              trackSelector=lambda x:[True]*len(x), 
              pairSelector=lambda x:[True]*len(x),
              verbose:bool=False):
    """ Loop over data to fill the invariant mass histogram.
        
        :param: dataDescription: is anything uproot.iterate can take.
                typical something like run*.data.root:eventsTree in our case
        :param: eventSelector: returns an array of bool from an array of events
        :param: trackSelector: returns an array of bool from an array of tracks
    """
    
    for batch in uproot.iterate(dataDescription,
                                ["isCINT","isCMUL","isCMSL","Muon_Px","Muon_Py","Muon_Pz","Muon_E","Muon_Charge","Muon_thetaAbs","Muon_matchedTrgThreshold"],                                
                                 step_size="100 MB", report=True):
        events=batch[0] # batch[1] is the report info
        if len(events) < 1000:
            print("something is wrong",batch[1]) # this is a protection for some corrupted input data files 
            break
        goodEvents = events[eventSelector(events)] 
        
        tracks = getTracks(goodEvents)
        goodTracks=tracks[trackSelector(tracks)]
    
        hMag.fill(ak.flatten(goodTracks.p))
        hTheta.fill(ak.flatten(goodTracks.theta))
        hEta.fill(ak.flatten(goodTracks.eta))
        hPt.fill(ak.flatten(goodTracks.pt))
        hPhi.fill(ak.flatten(goodTracks.phi))    
        
        pairs = getPairs(goodTracks)
        goodPairs = pairs[pairSelector(pairs)]
        
        hpMinv.fill(ak.flatten(goodPairs.mass))
        hpY.fill(ak.flatten(goodPairs.y))
        hpPt.fill(ak.flatten(goodPairs.pt))

        if verbose:
            print(batch[1])


In [ ]:
%%time
## SINGLE MUON TRACK PLOTS
#No cuts
vhMagRaw = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=100,name='$|p|$'))
vhThetaRaw = hist.Hist(hist.axis.Regular(bins=100,start=2,stop=22/7,name='$\\theta$'))
vhEtaRaw = hist.Hist(hist.axis.Regular(bins=100,start=-6,stop=0,name='$\eta$'))
vhPtRaw = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=20,name='$p_T$'))
vhPhiRaw = hist.Hist(hist.axis.Regular(bins=200,start=-22/7,stop=22/7,name='$\phi$'))
#Event selection
vhMagEvSel = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=100,name='$|p|$'))
vhThetaEvSel = hist.Hist(hist.axis.Regular(bins=100,start=2,stop=22/7,name='$\\theta$'))
vhEtaEvSel = hist.Hist(hist.axis.Regular(bins=100,start=-6,stop=0,name='$\eta$'))
vhPtEvSel = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=20,name='$p_T$'))
vhPhiEvSel = hist.Hist(hist.axis.Regular(bins=200,start=-22/7,stop=22/7,name='$\phi$'))
#Cuts applied
vhMag = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=100,name='$|p|$'))
vhTheta = hist.Hist(hist.axis.Regular(bins=100,start=2,stop=22/7,name='$\\theta$'))
vhEta = hist.Hist(hist.axis.Regular(bins=100,start=-6,stop=0,name='$\eta$'))
vhPt = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=20,name='$p_T$'))
vhPhi = hist.Hist(hist.axis.Regular(bins=200,start=-22/7,stop=22/7,name='$\phi$'))

## DIMUON MASS PAIR PLOTS
#No cuts
vhpMinvRaw = hist.Hist(hist.axis.Regular(bins=200,start=0,stop=20,name='$M_{\mu\mu}$'))
vhpYRaw = hist.Hist(hist.axis.Regular(bins=100,start=-7,stop=0,name='$y$'))
vhpPtRaw = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=20,name='$p_{T}$'))
#Event selection
vhpMinvEvSel = hist.Hist(hist.axis.Regular(bins=200,start=0,stop=20,name='$M_{\mu\mu}$'))
vhpYEvSel = hist.Hist(hist.axis.Regular(bins=100,start=-7,stop=0,name='$y$'))
vhpPtEvSel = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=20,name='$p_{T}$'))
#Cuts applied
vhpMinv = hist.Hist(hist.axis.Regular(bins=200,start=0,stop=20,name='$M_{\mu\mu}$'))
vhpY = hist.Hist(hist.axis.Regular(bins=100,start=-7,stop=0,name='$y$'))
vhpPt = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=20,name='$p_{T}$'))

scan(dataDescription=f"{BASEDIR}/run*.{DATATYPE}.root:eventsTree",
          hMag=vhMagRaw, hTheta=vhThetaRaw, hEta=vhEtaRaw, hPt=vhPtRaw, hPhi=vhPhiRaw,
          hpMinv=vhpMinvRaw, hpY=vhpYRaw, hpPt=vhpPtRaw,
          pairSelector=lambda x: x["unlike-sign"]==True)


scan(dataDescription=f"{BASEDIR}/run*.{DATATYPE}.root:eventsTree",
          hMag=vhMagEvSel, hTheta=vhThetaEvSel, hEta=vhEtaEvSel, hPt=vhPtEvSel, hPhi=vhPhiEvSel,
          hpMinv=vhpMinvEvSel, hpY=vhpYEvSel, hpPt=vhpPtEvSel,
          eventSelector=lambda x: x["isCMUL"]==True,
          pairSelector=lambda x: x["unlike-sign"]==True)

scan(dataDescription=f"{BASEDIR}/run*.{DATATYPE}.root:eventsTree",
          hMag=vhMag, hTheta=vhTheta, hEta=vhEta, hPt=vhPt, hPhi=vhPhi,
          hpMinv=vhpMinv, hpY=vhpY, hpPt=vhpPt,
          trackSelector=lambda x: (x["matched"]>0) & (x.pt>0) & (x.eta>-4) & (x.eta<-2.5) & (x.thetaAbs>2) & (x.thetaAbs<10),
          eventSelector=lambda x: x["isCMUL"]==True,
          pairSelector=lambda x: (x["y"] > -4) & (x["y"] < -2.5) & x["unlike-sign"]==True)

In [ ]:
vhMagRaw.plot(label="no cuts")
vhMagEvSel.plot(label="events selected")
vhMag.plot(label="tracks selected")
plt.yscale("log")
plt.ylabel("# of tracks")
plt.title('Total momentum of tracks');
plt.legend();


In [ ]:
vhThetaRaw.plot(label="no cuts")
vhThetaEvSel.plot(label="events selected")
vhTheta.plot(label="tracks selected")
plt.yscale("log")
plt.ylabel("# of tracks")

In [ ]:
vhEtaRaw.plot(label="no cuts")
vhEtaEvSel.plot(label="events selected")
vhEta.plot(label="tracks selected")
plt.yscale("log")
plt.ylabel("# of tracks")
plt.title('pseudorapidity of tracks');
plt.legend();

In [ ]:
vhPtRaw.plot(label="no cuts")
vhPtEvSel.plot(label="events selected")
vhPt.plot(label="tracks selected")
plt.yscale("log")
plt.ylabel("# of tracks")
plt.title('Transverse momentum of tracks');
plt.legend();

In [ ]:
vhPhiRaw.plot(label="no cuts")
vhPhiEvSel.plot(label="events selected")
vhPhi.plot(label="tracks selected")
plt.yscale("log")
plt.ylabel("# of tracks")
plt.title('Azimuthal angle distribution of tracks');
plt.legend();

# 2-Analyse en masse invariante des paires de muons

In [ ]:
vhpMinvRaw.plot(label="no cuts")
vhpMinvEvSel.plot(label="events selected")
vhpMinv.plot(label="tracks & pairs selected")
plt.yscale("log")
plt.ylabel("# of pairs")
plt.title('Invariant mass distribution of muons');
plt.legend();

In [ ]:
vhpYRaw.plot(label="no cuts")
vhpYEvSel.plot(label="events selected")
vhpY.plot(label="tracks & pairs selected")
plt.yscale("log")
plt.ylabel("# of pairs")
plt.title('Rapidity distribution of muons');
plt.legend();

In [ ]:
vhpPtRaw.plot(label="no cuts")
vhpPtEvSel.plot(label="events selected")
vhpPt.plot(label="tracks & pairs selected")
plt.yscale("log")
plt.ylabel("# of pairs")
plt.title('Transverse momentum distribution of muons');
plt.legend();

In [ ]:
with uproot.recreate("all.dimuHist.root") as file:
    file["hMinv"]=vhpMinv
    file["hY"]=vhpY
    file["hPt"]=vhpPt

In [ ]:
with uproot.open("all.hminv.root") as file:
    print(file.keys())
    h1=file["hMinv"].to_hist()
    h2=file["hY"].to_hist()
    h3=file["hPt"].to_hist()

In [ ]:
h1.plot()

In [ ]:
h2.plot()

In [ ]:
h3.plot()